In [ ]:
# %pip install numpy pandas xgboost lightgbm scikit-learn openpyxl

In [ ]:
# %pip install cupy-cuda12x

In [ ]:
# %pip install torch torchvision torchaudio

In [ ]:
import numpy as np
import pandas as pd

from utils.utils import load_dataset
from utils.workflow import train_eval, case_study

from function.ITRPCA import ITRPCA_F
from function.VDA_GKSBMF import A_VDA_GMSBMF
from function.DRPADC import WeightImputeLogFactorization, GetProbability
from function.MVL import MVL_F
from function.BMC import BMC_F
from function.MSBMF import MSBMF_F
from function.HGIMC import fBMC, fGRB, fHGI
from function.AdaMVL import AdaMVL

## HN-DREP Benchmark

In [ ]:
benchmark = pd.read_excel('data/Benchmark/evaluation-results.xlsx')
benchmark.head()

### Note: Explanation of High Average F1 Score in the HINGRL Algorithm

The HINGRL algorithm achieved a relatively high average F1 score (0.8789) on the Fdataset dataset. However, it’s important to note that this high score may be influenced by specific strategies in generating negative samples and the model training methods used. Key points are outlined below:

1. **Negative Sample Generation Strategy:**
   - The algorithm generates an equal number of negative samples by randomly selecting drug-disease pairs that are not labeled as positive in the existing dataset. While this ensures a balanced number of positive and negative samples, the generated negatives may not represent true non-associated pairs—they could be potential positives that are simply undiscovered.
   - This simple random sampling method does not account for biological similarities, functional annotations, or other prior knowledge, potentially making the negatives less representative of real-world scenarios. As a result, the model may perform well on the training set but have limited generalization capability in real applications.

2. **Balanced Data Leading to Inflated F1 Scores:**
   - The training set is perfectly balanced with equal positive and negative samples, which makes it easier for the model to achieve high precision and recall, and thus a higher F1 score.
   - In real-world applications, negative samples usually vastly outnumber positive ones. This artificially balanced training data does not reflect real-world distributions, causing the F1 score to be overestimated.

3. **Use of Random Forest for Training:**
   - The HINGRL algorithm utilizes a Random Forest model, which performs exceptionally well on balanced datasets by leveraging multiple decision trees to reduce overfitting and improve precision and recall.
   - While it achieves high F1 scores on balanced data, its performance may degrade on real, imbalanced datasets where the proportion of negative samples is much higher.

**Conclusion:**

The high average F1 score achieved by the HINGRL algorithm is partly due to the simplistic negative sample generation and training on a balanced dataset. While the results appear promising on the current dataset, its generalization to more complex and imbalanced real-world drug-disease prediction scenarios may be limited.

## Data
- **Fdataset**
- **Cdataset**
- **Ydataset**

Each of the four algorithms achieved the highest score on the datasets mentioned above.

In [ ]:
dataset_name = 'Fdataset'
drug_name, disease_name, Wrd, Wrr, Wrr_six_geps, Wrr_six_kgs, Wrr_six_llms, Wrr_five, Wdd, Wdd_two, Trr, Trr_six_geps, Trr_six_kgs, Trr_six_llms, Trr_five, Tdd, Tdd_two, drug_embeddings, disease_embeddings = load_dataset(dataset_name, embedding_type='llm')

In [ ]:
drug_name[:3], disease_name[:3]

In [ ]:
# Wrr = [drug_ChemS, drug_AtcS, drug_SideS, drug_DDIS, drug_TargetS, drug_GepS]
# Wdd = [disease_PhS, disease_DoS]
len(Wrr), len(Wdd), len(Wrr_five), len(Wdd_two)

In [ ]:
len(Wrr_six_llms), Trr_six_llms.shape

In [ ]:
Wrd.shape, Wrr[0].shape, Wdd[0].shape, Trr.shape, Tdd.shape, Wrr_five[0].shape, Trr_five.shape

In [ ]:
drug_embeddings.shape, disease_embeddings.shape

## AdaMVL

### Baseline

In [ ]:
# 5 + 2
train_eval(
    algorithm_func=AdaMVL,
    Wrd=Wrd,
    Wrr_list=Wrr_five,
    Wdd_list=Wdd_two,
    Trr=Trr_five,
    Tdd=Tdd_two,
    folds=10
)

### 8+3

In [ ]:
# 8 + 3
train_eval(
    algorithm_func=AdaMVL,
    Wrd=Wrd,
    Wrr_list=Wrr,
    Wdd_list=Wdd,
    Trr=Trr,
    Tdd=Tdd,
    folds=10
)

### 8+2

In [ ]:
# 8 + 2
train_eval(
    algorithm_func=AdaMVL,
    Wrd=Wrd,
    Wrr_list=Wrr,
    Wdd_list=Wdd_two,
    Trr=Trr,
    Tdd=Tdd_two,
    folds=10
)

### 6+2

In [ ]:
# 6(llms) + 2
train_eval(
    algorithm_func=AdaMVL,
    Wrd=Wrd,
    Wrr_list=Wrr_six_llms,
    Wdd_list=Wdd_two,
    Trr=Trr_six_llms,
    Tdd=Tdd_two,
    folds=10
)

In [ ]:
# 6(kgs) + 2
train_eval(
    algorithm_func=AdaMVL,
    Wrd=Wrd,
    Wrr_list=Wrr_six_kgs,
    Wdd_list=Wdd_two,
    Trr=Trr_six_kgs,
    Tdd=Tdd_two,
    folds=10
)

In [ ]:
# 6(geps) + 2
train_eval(
    algorithm_func=AdaMVL,
    Wrd=Wrd,
    Wrr_list=Wrr_six_geps,
    Wdd_list=Wdd_two,
    Trr=Trr_six_geps,
    Tdd=Tdd_two,
    folds=10
)

### Case Study

In [ ]:
case_study(Wrd, Wrr, Wdd, Trr, Tdd, algorithm_func=AdaMVL, drug_names=drug_name, disease_names=disease_name, top=100)

## MLMC

In [ ]:
def MLMC(Wrr_list, Wdd_list, Wrd):
    alphaBMC = 10
    betaBMC = 10
    thresholdBMC = 0.8
    maxiterBMC = 300
    tol1BMC = 2 * 1e-3
    tol2BMC = 1 * 1e-5

    Wrr_ML = [w.copy() for w in Wrr_list]
    Wdd_ML = [w.copy() for w in Wdd_list]
    
    for i in range(len(Wrr_ML)):
        np.fill_diagonal(Wrr_ML[i], 0)
    
    for i in range(len(Wdd_ML)):
        np.fill_diagonal(Wdd_ML[i], 0)

    _, _, F = MVL_F(Wrr_ML, Wdd_ML, Wrd, 0.1, 0.1)

    trIndexBMC = (Wrd.T != 0).astype(float)
    A_bmc, iter = BMC_F(alphaBMC, betaBMC, Wrd.T, trIndexBMC, tol1BMC, tol2BMC, maxiterBMC, 0, 1)
    Wdr0 = A_bmc * (A_bmc > thresholdBMC)
    SR_MC, SD_MC, F_MC = MVL_F(Wrr_ML, Wdd_ML, Wdr0.T, 0.1, 0.1)

    return np.maximum(F, F_MC)

In [ ]:
# 5 + 2
train_eval(Wrd, Wrr_five, Wdd_two, Trr_five, Tdd_two, MLMC)

In [ ]:
# 8 + 3
train_eval(Wrd, Wrr, Wdd, Trr, Tdd, MLMC)

In [ ]:
# 8 + 2
train_eval(Wrd, Wrr, Wdd_two, Trr, Tdd_two, MLMC)

In [ ]:
# 6(llms) + 2
train_eval(Wrd, Wrr_six_llms, Wdd_two, Trr_six_llms, Tdd_two, MLMC)

In [ ]:
# 6(kgs) + 2
train_eval(Wrd, Wrr_six_kgs, Wdd_two, Trr_six_kgs, Tdd_two, MLMC)

In [ ]:
# 6(geps) + 2
train_eval(Wrd, Wrr_six_geps, Wdd_two, Trr_six_geps, Tdd_two, MLMC)

## MSBMF

In [ ]:
def MSBMF(Wrr_list, Wdd_list, Wrd):
    lambda1 = 0.1
    lambda2 = 0.01
    lambda3 = lambda2
    k = int(min(Wrd.shape) * 0.7)
    maxiter = 300
    tol1 = 2 * 1e-3
    tol2 = 1 * 1e-4

    # Wrr = [Wrr1, Wrr2, Wrr3, Wrr4, Wrr5];
    Wrr = np.hstack(Wrr_list)
    # Wdd = [Wdd1, Wdd2];
    Wdd = np.hstack(Wdd_list)

    U, V, iter = MSBMF_F(Wrd.T, Wdd, Wrr, lambda1, lambda2, lambda3, k, tol1, tol2, maxiter);
    M_recovery = U @ V.T

    return M_recovery.T

In [ ]:
# 5 + 2
train_eval(Wrd, Wrr_five, Wdd_two, Trr_five, Tdd_two, MSBMF)

In [ ]:
# 8 + 3
train_eval(Wrd, Wrr, Wdd, Trr, Tdd, MSBMF)

In [ ]:
# 8 + 2
train_eval(Wrd, Wrr, Wdd_two, Trr, Tdd_two, MSBMF)

In [ ]:
# 6(llms) + 2
train_eval(Wrd, Wrr_six_llms, Wdd_two, Trr_six_llms, Tdd_two, MSBMF)

In [ ]:
# 6(kgs) + 2
train_eval(Wrd, Wrr_six_kgs, Wdd_two, Trr_six_kgs, Tdd_two, MSBMF)

In [ ]:
# 6(geps) + 2
train_eval(Wrd, Wrr_six_geps, Wdd_two, Trr_six_geps, Tdd_two, MSBMF)

## ITRPCA

In [ ]:
def ITRPCA(Wrd, Trr, Tdd):
    p = 0.9
    K = 30
    rat1 = 0.1
    rat2 = 0.2

    F_ITRPCA = ITRPCA_F(Trr, Tdd, Wrd.T, p, K, rat1, rat2).T

    return F_ITRPCA

In [ ]:
# 5 + 2
train_eval(Wrd, Wrr_five, Wdd_two, Trr_five, Tdd_two, ITRPCA)

In [ ]:
# 8 + 3
train_eval(Wrd, Wrr, Wdd, Trr, Tdd, ITRPCA)

In [ ]:
# 8 + 2
train_eval(Wrd, Wrr, Wdd_two, Trr, Tdd_two, ITRPCA)

In [ ]:
# 6(llms) + 2
train_eval(Wrd, Wrr_six_llms, Wdd_two, Trr_six_llms, Tdd_two, ITRPCA)

In [ ]:
# 6(kgs) + 2
train_eval(Wrd, Wrr_six_kgs, Wdd_two, Trr_six_kgs, Tdd_two, ITRPCA)

In [ ]:
# 6(geps) + 2
train_eval(Wrd, Wrr_six_geps, Wdd_two, Trr_six_geps, Tdd_two, ITRPCA)

## HGIMC

In [ ]:
def HGIMC(Wrr_list, Wdd_list, Wrd):
    A_DR = Wrd.T

    # # Base
    # R = Wrr_list[0]
    # D = Wdd_list[1]

    # Average
    R = np.mean(Wrr_list, axis=0)
    D = np.mean(Wdd_list, axis=0)

    alpha = 10
    beta = 10
    gamma = 0.1
    threshold = 0.1
    maxiter = 300
    tol1 = 2 * 1e-3
    tol2 = 1 * 1e-5

    trIndex = (A_DR != 0).astype(float)
    A_bmc, iter = fBMC(alpha, beta, A_DR, trIndex, tol1, tol2, maxiter, 0, 1)
    A_DR0 = A_bmc * (A_bmc > threshold)

    A_RR = fGRB(R, 0.5)
    A_DD = fGRB(D, 0.5)

    A_recovery = fHGI(gamma, A_DD, A_RR, A_DR0)

    return A_recovery.T


In [ ]:
# 5 + 2
train_eval(Wrd, Wrr_five, Wdd_two, Trr_five, Tdd_two, HGIMC)

In [ ]:
# 8 + 3
train_eval(Wrd, Wrr, Wdd, Trr, Tdd, HGIMC)

In [ ]:
# 8 + 2
train_eval(Wrd, Wrr, Wdd_two, Trr, Tdd_two, HGIMC)

In [ ]:
# 6(llms) + 2
train_eval(Wrd, Wrr_six_llms, Wdd_two, Trr_six_llms, Tdd_two, HGIMC)

In [ ]:
# 6(kgs) + 2
train_eval(Wrd, Wrr_six_kgs, Wdd_two, Trr_six_kgs, Tdd_two, HGIMC)

In [ ]:
# 6(geps) + 2
train_eval(Wrd, Wrr_six_geps, Wdd_two, Trr_six_geps, Tdd_two, HGIMC)

## DRPADC

In [ ]:
def DRPADC(Wrd, Wrr_list, Wdd_list):
    rank = 150
    rnseed = 0
    lR = 0.1
    lM = 1.0
    lN = 0.1
    num_iter = 550
    learn_rate = 0.09

    W = np.maximum(1, Wrd)

    # # Paper Setting
    # Wdd = Wdd_list[1]  # semantic sim
    # Wrr = Wrr_list[0]  # chemical sim
    
    # Benchmark
    Wdd = np.mean(Wdd_list, axis=0)
    Wrr = np.mean(Wrr_list, axis=0)

    F, G = WeightImputeLogFactorization(Wrd, Wrr, Wdd, W, rank, lR, lM, lN, num_iter, learn_rate, rnseed)
    
    PROB = GetProbability(np.dot(F, G.T))
    
    return PROB

In [ ]:
# 5 + 2
train_eval(Wrd, Wrr_five, Wdd_two, Trr_five, Tdd_two, DRPADC)

In [ ]:
# 8 + 3
train_eval(Wrd, Wrr, Wdd, Trr, Tdd, DRPADC)

In [ ]:
# 8 + 2
train_eval(Wrd, Wrr, Wdd_two, Trr, Tdd_two, DRPADC)

In [ ]:
# 6(llms) + 2
train_eval(Wrd, Wrr_six_llms, Wdd_two, Trr_six_llms, Tdd_two, DRPADC)

In [ ]:
# 6(kgs) + 2
train_eval(Wrd, Wrr_six_kgs, Wdd_two, Trr_six_kgs, Tdd_two, DRPADC)

In [ ]:
# 6(geps) + 2
train_eval(Wrd, Wrr_six_geps, Wdd_two, Trr_six_geps, Tdd_two, DRPADC)

## VDA-GKSBMF

In [ ]:
def VDA_GKSBMF(Wrd, Wrr_list, Wdd_list):
    gm = 0.5
    w = 0.3
    lambda1 = 1
    lambda2 = lambda1
    lambda3 = lambda2
    tol1 = 2 * 1e-30
    tol2 = 2 * 1e-40
    maxiter = 400

    k = int(min(Wrd.shape) * 0.7)

    # # Paper Setting
    # Wdd = Wdd_list[1]
    # Wrr = Wrr_list[0]

    # Benchmark
    Wdd = np.mean(Wdd_list, axis=0)
    Wrr = np.mean(Wrr_list, axis=0)

    M_recovery = A_VDA_GMSBMF(Wrd.T, Wdd, Wrr, gm, w, lambda1, lambda2, lambda3, k, tol1, tol2, maxiter)
    
    return M_recovery.T

In [ ]:
# 5 + 2
train_eval(Wrd, Wrr_five, Wdd_two, Trr_five, Tdd_two, VDA_GKSBMF)

In [ ]:
# 8 + 3
train_eval(Wrd, Wrr, Wdd, Trr, Tdd, VDA_GKSBMF)

In [ ]:
# 8 + 2
train_eval(Wrd, Wrr, Wdd_two, Trr, Tdd_two, VDA_GKSBMF)

In [ ]:
# 6(llms) + 2
train_eval(Wrd, Wrr_six_llms, Wdd_two, Trr_six_llms, Tdd_two, VDA_GKSBMF)

In [ ]:
# 6(kgs) + 2
train_eval(Wrd, Wrr_six_kgs, Wdd_two, Trr_six_kgs, Tdd_two, VDA_GKSBMF)

In [ ]:
# 6(geps) + 2
train_eval(Wrd, Wrr_six_geps, Wdd_two, Trr_six_geps, Tdd_two, VDA_GKSBMF)

## Machine Learning

In [ ]:
train_eval(
    drug_embeddings=drug_embeddings,
    disease_embeddings=disease_embeddings,
    Wrd=Wrd,
    Wrr_list=Wrr,
    Wdd_list=Wdd,
    Trr=Trr,
    Tdd=Tdd,
    ml_benchmark=True
)